In [1]:
%load_ext Cython
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from SuchTree import SuchTree, SuchLinkedTrees
import pandas as pd
import numpy as np
import seaborn

/home/russell/opt/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
T1 = SuchTree( 'SuchTree/tests/test.tree' )
T2 = SuchTree( 'http://edhar.genomecenter.ucdavis.edu/~russell/fishpoo/fishpoo2_p200_c2_unique_2_clustalo_fasttree.tree' )
links = pd.read_csv( 'http://edhar.genomecenter.ucdavis.edu/~russell/fishpoo/fishpoo2_p200_c2_host_count_table.tsv', 
                    sep='\t', index_col='Host')
links.index = map( lambda x : x.replace(' ','_'), links.index )

In [ ]:
T1 = SuchTree( 'SuchTree/tests/test.tree' )
T2 = SuchTree( 'SuchTree/tests/test.tree' )
links = pd.DataFrame( np.random.random_integers( 0, 3, size=(14,14) ), index=T1.leafs.keys(), columns=T1.leafs.keys() )

In [4]:
SLT = SuchLinkedTrees( T1, T2, links )
#SLT = SuchLinkedTrees( T1, T2, links )

139993375484744 allocating columns in 3211438944
bulding default subset.
bulding default link list.


In [ ]:
SLT.dump_table()

In [ ]:
SLT.linkmatrix

In [ ]:
SLT.linklist[0,0]

In [ ]:
result = SLT.linked_distances()

seaborn.jointplot( result['TreeA'], result['TreeB'] )

In [ ]:
SLT.TreeB.get_internal_nodes()

In [ ]:
SLT.TreeB.get_leafs( 7027 )

In [ ]:
SLT.subset( 7027 )
print SLT.subset_size, SLT.subset_n_links

In [ ]:
print SLT.col_ids
print SLT.subset_columns
print SLT.subset_leafs

In [ ]:
SLT.linklist

In [ ]:
size = (SLT.subset_size*(SLT.subset_size-1))/2

ids_a = zeros( (size, 2), dtype=int )
ids_b = zeros( (size, 2), dtype=int )
k=0
for i in xrange( SLT.subset_size ) :
    for j in xrange( i ) :
        ids_a[ k, 0 ] = SLT.linklist[ i, 1 ]
        ids_a[ k, 1 ] = SLT.linklist[ j, 1 ]
        ids_b[ k, 0 ] = SLT.linklist[ i, 0 ]
        ids_b[ k, 1 ] = SLT.linklist[ j, 0 ]
        k += 1

In [ ]:
result = SLT.linked_distances()

seaborn.jointplot( result['TreeA'], result['TreeB'] )

In [ ]:
SLT.get_column_leafs( 0, as_row_ids=True )

In [ ]:
SLT.get_column_links( 0 )

In [ ]:
s = links[ links.index[0] ]
map( list(SLT.col_ids).index, map( lambda x : T1.leafs[x],  s[ s > 0 ].index ) )

In [ ]:
set((1,2,3)) == set((1,2,3))

In [ ]:
for a,b, in zip( s.index, SLT.row_names ) :
    print a == b

In [ ]:
for n in xrange( 14 ) :
    for a,b in zip( SLT.linkmatrix[0], SLT.get_column_links(0) ) :
        print a == b

In [ ]:
for col in SLT.col_names :
    for row in SLT.row_names :
        col_id = SLT.col_names.index(col)
        row_id = SLT.row_names.index(row)
        print bool(links[col][row]) == SLT.linkmatrix[col_id][row_id]

In [ ]:
%%cython

cdef extern from "stdlib.h":
    long int random()

def foo_rand() :
    return random()

In [ ]:
foo_rand()

In [ ]:
random

In [ ]:
%%cython
from numpy.random import randint
cdef extern from 'stdint.h' :
    ctypedef unsigned long int uint64_t
    uint64_t UINT64_MAX

cdef class rng :
    cdef uint64_t x
    cdef uint64_t a
    
    def __cinit__( self ) :
        #self.x = randint( UINT64_MAX >> 1 )
        self.x = 1
        self.a = 2685821657736338717
     
    cdef unsigned long _xorshift64star( self ) nogil :
        self.x ^= self.x >> 12 # a
        self.x ^= self.x << 25 # b
        self.x ^= self.x >> 27 # c
        return self.x * self.a

    def xorshiftstar( self ) :
        return self._xorshift64star()
    
    def random_int( self, uint64_t n ) :
        return self._xorshift64star() % n

In [ ]:
R = rng()
print R.xorshiftstar()
print R.xorshiftstar()
print R.xorshiftstar()
print R.xorshiftstar()
print R.xorshiftstar()
print R.xorshiftstar()

#randint( 18446744073709551615 >> 1 )

In [ ]:
b = []
for i in xrange( 10000 ) :
    b.append(R.xorshiftstar( 100 ))

In [ ]:
SLT.linklist

In [11]:
SLT.sample_linked_distances(sigma=0.05, n=4000, buckets=10)

0.133099406958 0.00433306628838
0.102310940623 0.00471829809248
0.0690533965826 0.00464188307524
0.0669212564826 0.00403016945347
0.0471185073256 0.00332739576697


{'n': 200000, 'r': 0.007994653677467312}

In [ ]:
for n,s in enumerate(links.iteritems()) :
    if n > 10 : break
    print s

In [ ]:
links.columns[1]

In [ ]:
%%cython
from SuchTree import SuchTree
from itertools import combinations
from numpy import array, std

T1 = SuchTree( 'SuchTree/tests/test.tree' )

cdef double [:] d_mv

pairs = []
for a,b in combinations( T1.leafs.values(), 2 ) :
    pairs.append( [a,b] )
d = T1.distances( array(pairs) )
d_mv = d
print std(d_mv)

In [ ]:
( mean(d**2) - mean(d)**2 )**(0.5)